In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
anzsic_path = './data/anzsic_codes_and_titles.xls'

In [4]:
df = pd.read_excel(anzsic_path, sheet_name=4, skiprows=5)

In [5]:
new_columns = ['None',
               'DivisionCode',
               'SubdivisionCode',
               'GroupCode',
               'ClassCode',
               'ClassName'
              ]

In [6]:
df.columns=  new_columns
df.head()

,None,DivisionCode,SubdivisionCode,GroupCode,ClassCode,ClassName
0,NaN,A,"Agriculture, Forestry and Fishing",NaN,NaN,NaN
1,NaN,NaN,01,Agriculture,NaN,NaN
2,NaN,NaN,NaN,011,Nursery and Floriculture Production,NaN
3,NaN,NaN,NaN,NaN,0111,Nursery Production (Under Cover)
4,NaN,NaN,NaN,NaN,0112,Nursery Production (Outdoors)


This is a disgusting way of doing it, but let's iterate over rows

In [7]:
class_list = []

current_div_code = None
current_div_name = None
current_subdiv_code = None
current_subdiv_name = None
current_group_code = None
current_group_name = None

for index, row in df.iterrows():
    if not pd.isna(row['DivisionCode']):
        # New division
        current_div_code = row['DivisionCode']
        current_div_name = row['SubdivisionCode']
        continue
    if not pd.isna(row['SubdivisionCode']) and len(row['SubdivisionCode'])==2:
        # New subdivision
        current_subdiv_code = row['SubdivisionCode']
        current_subdiv_name = row['GroupCode']
        continue
    if not pd.isna(row['GroupCode']) and len(row['GroupCode'])==3:
        # New group
        current_group_code = row['GroupCode']
        current_group_name = row['ClassCode']
        continue
    
    if not pd.isna(row['ClassCode']) and not pd.isna(row['ClassName']):
        class_list.append([current_div_code,
                          current_div_name,
                          current_subdiv_code,
                          current_subdiv_name,
                          current_group_code,
                          current_group_name,
                          row['ClassCode'],
                          row['ClassName']])
    
        
    

In [8]:
# This dataframe contains a different class for each row
# Columns describe div,subdiv,group names and codes
column_names = columns=['DivisionCode',
                             'DivisionName',
                             'SubDivisionCode',
                             'SubDivisionName',
                             'GroupCode',
                             'GroupName',
                             'ClassCode',
                             'ClassName']
df_anzsic = pd.DataFrame(data=class_list,
                     columns=column_names)
df_anzsic.to_csv('anzsic_df.csv')
df_anzsic.head()


,DivisionCode,DivisionName,SubDivisionCode,SubDivisionName,GroupCode,GroupName,ClassCode,ClassName
0,A,"Agriculture, Forestry and Fishing",01,Agriculture,011,Nursery and Floriculture Production,0111,Nursery Production (Under Cover)
1,A,"Agriculture, Forestry and Fishing",01,Agriculture,011,Nursery and Floriculture Production,0112,Nursery Production (Outdoors)
2,A,"Agriculture, Forestry and Fishing",01,Agriculture,011,Nursery and Floriculture Production,0113,Turf Growing
3,A,"Agriculture, Forestry and Fishing",01,Agriculture,011,Nursery and Floriculture Production,0114,Floriculture Production (Under Cover)
4,A,"Agriculture, Forestry and Fishing",01,Agriculture,011,Nursery and Floriculture Production,0115,Floriculture Production (Outdoors)


In [9]:
# This dict maps from a 4-digit ANZSIC code (as a string) to a dict of the div,subdiv,group names and codes
dict_anzsic = { x[6]: {column_names[i]:y for i,y in enumerate(x)} for x in class_list}

with open('anzsic_dict.pkl','wb') as f:
    pickle.dump(dict_anzsic, f)

dict_anzsic['1334']

{'DivisionCode': 'C',
 'DivisionName': 'Manufacturing',
 'SubDivisionCode': '13',
 'SubDivisionName': 'Textile, Leather, Clothing and Footwear Manufacturing',
 'GroupCode': '133',
 'GroupName': 'Textile Product Manufacturing',
 'ClassCode': '1334',
 'ClassName': 'Textile Finishing and Other Textile Product Manufacturing'}

In [10]:
dict_anzsic['1']

KeyError: '1'